# Experiment 1: Optimize AUROC using AUCMLoss + PESG

In [1]:
# install libauc
!pip install -U libauc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# import required libraries
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from libauc.datasets import CIFAR10
from libauc.utils import ImbalancedDataGenerator
from libauc.sampler import DualSampler

# set seed and device
def set_all_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(2023)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# dataset class with augmentation
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=32, crop_size=30, mode='train'):
        self.images = images.astype(np.uint8)
        self.targets = targets
        self.mode = mode
        self.transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.RandomCrop((crop_size, crop_size)),
            transforms.RandomHorizontalFlip(),
            transforms.Resize((image_size, image_size))
        ])
        self.transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((image_size, image_size))
        ])
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, self.targets[idx], idx


In [4]:
# load and imbalance cifar10
train_data, train_targets = CIFAR10(root='./data', train=True).as_array()
test_data, test_targets  = CIFAR10(root='./data', train=False).as_array()

generator = ImbalancedDataGenerator(verbose=True, random_seed=2023)
train_images, train_labels = generator.transform(train_data, train_targets, imratio=0.02)
test_images, test_labels = generator.transform(test_data, test_targets, imratio=0.5)

trainSet = ImageDataset(train_images, train_labels)
trainSet_eval = ImageDataset(train_images, train_labels, mode='test')
testSet = ImageDataset(test_images, test_labels, mode='test')



100%|██████████| 170M/170M [00:03<00:00, 45.7MB/s]


Files already downloaded and verified
#SAMPLES: 25510, CLASS 0.0 COUNT: 25000, CLASS RATIO: 0.9800
#SAMPLES: 25510, CLASS 1.0 COUNT: 510, CLASS RATIO: 0.0200
#SAMPLES: 10000, CLASS 0.0 COUNT: 5000, CLASS RATIO: 0.5000
#SAMPLES: 10000, CLASS 1.0 COUNT: 5000, CLASS RATIO: 0.5000


In [5]:
from libauc.models import resnet20 as ResNet
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.metrics import auc_roc_score

BATCH_SIZE = 128
lr = 0.1
margin = 1.0
epoch_decay = 0.003
weight_decay = 1e-4
total_epochs = 30
decay_epochs = [15, 25]

model = ResNet(pretrained=False, last_activation=None, num_classes=1).to(device)
loss_fn = AUCMLoss()
optimizer = PESG(model.parameters(), loss_fn=loss_fn, lr=lr,
                 margin=margin, epoch_decay=epoch_decay, weight_decay=weight_decay)

sampler = DualSampler(trainSet, BATCH_SIZE, sampling_rate=0.2)
trainloader = torch.utils.data.DataLoader(trainSet, batch_size=BATCH_SIZE, sampler=sampler)
testloader = torch.utils.data.DataLoader(testSet, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# train and evaluate auroc
for epoch in range(total_epochs):
    if epoch in decay_epochs:
        optimizer.update_regularizer(decay_factor=10)

    model.train()
    for data, targets, _ in trainloader:
        data, targets = data.to(device), targets.to(device)
        y_pred = torch.sigmoid(model(data))
        loss = loss_fn(y_pred, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for data, targets, _ in testloader:
            data = data.to(device)
            out = torch.sigmoid(model(data))
            preds.append(out.cpu().numpy())
            labels.append(targets.numpy())
    auc = auc_roc_score(np.concatenate(labels), np.concatenate(preds))
    print(f"[Epoch {epoch}] AUROC: {auc:.4f}")

[Epoch 0] AUROC: 0.5994
[Epoch 1] AUROC: 0.6946
[Epoch 2] AUROC: 0.6841
[Epoch 3] AUROC: 0.6595
[Epoch 4] AUROC: 0.7097
[Epoch 5] AUROC: 0.7033
[Epoch 6] AUROC: 0.7195
[Epoch 7] AUROC: 0.7316
[Epoch 8] AUROC: 0.7266
[Epoch 9] AUROC: 0.7129
[Epoch 10] AUROC: 0.6990
[Epoch 11] AUROC: 0.7205
[Epoch 12] AUROC: 0.7156
[Epoch 13] AUROC: 0.7090
[Epoch 14] AUROC: 0.7350
Reducing learning rate to 0.01000 @ T=3630!
Updating regularizer @ T=3630!
[Epoch 15] AUROC: 0.7669
[Epoch 16] AUROC: 0.7639
[Epoch 17] AUROC: 0.7635
[Epoch 18] AUROC: 0.7597
[Epoch 19] AUROC: 0.7607
[Epoch 20] AUROC: 0.7555
[Epoch 21] AUROC: 0.7580
[Epoch 22] AUROC: 0.7569
[Epoch 23] AUROC: 0.7542
[Epoch 24] AUROC: 0.7534
Reducing learning rate to 0.00100 @ T=6050!
Updating regularizer @ T=6050!
[Epoch 25] AUROC: 0.7531
[Epoch 26] AUROC: 0.7539
[Epoch 27] AUROC: 0.7531
[Epoch 28] AUROC: 0.7529
[Epoch 29] AUROC: 0.7523


In [7]:
# Experiment 2: Optimize AUPRC using APLoss + SOAP

In [8]:
from libauc.models import resnet18 as ResNet
from libauc.losses import APLoss
from libauc.optimizers import SOAP
from libauc.metrics import auc_prc_score

BATCH_SIZE = 64
lr = 1e-3
margin = 0.6
gamma = 0.1
weight_decay = 2e-4
total_epochs = 30
decay_epochs = [15, 25]

model = ResNet(pretrained=False, last_activation=None, num_classes=1).to(device)
loss_fn = APLoss(data_len=len(trainSet), margin=margin, gamma=gamma)
optimizer = SOAP(model.parameters(), lr=lr, mode='adam', weight_decay=weight_decay)

sampler = DualSampler(trainSet, BATCH_SIZE, sampling_rate=0.5)
trainloader = torch.utils.data.DataLoader(trainSet, batch_size=BATCH_SIZE, sampler=sampler)
testloader = torch.utils.data.DataLoader(testSet, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
# train and evaluate auprc
for epoch in range(total_epochs):
    if epoch in decay_epochs:
        optimizer.update_lr(decay_factor=10)

    model.train()
    for data, targets, idx in trainloader:
        data, targets, idx = data.to(device), targets.to(device), idx.to(device)
        y_prob = torch.sigmoid(model(data))
        loss = loss_fn(y_prob, targets, idx)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for data, targets, _ in testloader:
            data = data.to(device)
            out = torch.sigmoid(model(data))
            preds.append(out.cpu().numpy())
            labels.append(targets.numpy())
    ap = auc_prc_score(np.concatenate(labels), np.concatenate(preds))
    print(f"[Epoch {epoch}] AUPRC: {ap:.4f}")



[Epoch 0] AUPRC: 0.6697
[Epoch 1] AUPRC: 0.6896
[Epoch 2] AUPRC: 0.7179
[Epoch 3] AUPRC: 0.7335
[Epoch 4] AUPRC: 0.7250
[Epoch 5] AUPRC: 0.6994
[Epoch 6] AUPRC: 0.7340
[Epoch 7] AUPRC: 0.7343
[Epoch 8] AUPRC: 0.7304
[Epoch 9] AUPRC: 0.7303
[Epoch 10] AUPRC: 0.7211
[Epoch 11] AUPRC: 0.7189
[Epoch 12] AUPRC: 0.7230
[Epoch 13] AUPRC: 0.6760
[Epoch 14] AUPRC: 0.7208
Reducing learning rate to 0.00010 @ T=11715!
[Epoch 15] AUPRC: 0.7343
[Epoch 16] AUPRC: 0.7256
[Epoch 17] AUPRC: 0.7299
[Epoch 18] AUPRC: 0.7169
[Epoch 19] AUPRC: 0.7164
[Epoch 20] AUPRC: 0.7198
[Epoch 21] AUPRC: 0.6604
[Epoch 22] AUPRC: 0.6700
[Epoch 23] AUPRC: 0.6995
[Epoch 24] AUPRC: 0.7013
Reducing learning rate to 0.00001 @ T=19525!
[Epoch 25] AUPRC: 0.7007
[Epoch 26] AUPRC: 0.6958
[Epoch 27] AUPRC: 0.6918
[Epoch 28] AUPRC: 0.6937
[Epoch 29] AUPRC: 0.6843


In [10]:
# 🔬 Experiment 3: Combine AUROC + AUPRC with r-weighted loss

In [11]:
from libauc.losses import AUCMLoss, APLoss
from torch.optim import Adam

r = 0.5  # try 0.2, 0.5, 0.8
model = ResNet(pretrained=False, last_activation=None, num_classes=1).to(device)
loss_ap = APLoss(data_len=len(trainSet), margin=0.6, gamma=0.1)
loss_auc = AUCMLoss()
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

trainloader = torch.utils.data.DataLoader(trainSet, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=False)

In [13]:
# train and evaluate combined loss
for epoch in range(30):
    model.train()
    for data, targets, idx in trainloader:
        data, targets, idx = data.to(device), targets.to(device), idx.to(device)

        # check if batch has at least one positive sample
        if (targets == 1).sum() == 0:
            continue  # skip this batch

        out = torch.sigmoid(model(data))
        loss = r * loss_ap(out, targets, idx) + (1 - r) * loss_auc(out, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for data, targets, _ in testloader:
            data = data.to(device)
            out = torch.sigmoid(model(data))
            preds.append(out.cpu().numpy())
            labels.append(targets.numpy())
    ap = auc_prc_score(np.concatenate(labels), np.concatenate(preds))
    print(f"[Epoch {epoch}] Combined AUPRC: {ap:.4f} (r={r})")


[Epoch 0] Combined AUPRC: 0.4798 (r=0.5)
[Epoch 1] Combined AUPRC: 0.5280 (r=0.5)
[Epoch 2] Combined AUPRC: 0.5869 (r=0.5)
[Epoch 3] Combined AUPRC: 0.6093 (r=0.5)
[Epoch 4] Combined AUPRC: 0.6061 (r=0.5)
[Epoch 5] Combined AUPRC: 0.6159 (r=0.5)
[Epoch 6] Combined AUPRC: 0.6249 (r=0.5)
[Epoch 7] Combined AUPRC: 0.6225 (r=0.5)
[Epoch 8] Combined AUPRC: 0.6330 (r=0.5)
[Epoch 9] Combined AUPRC: 0.6364 (r=0.5)
[Epoch 10] Combined AUPRC: 0.6397 (r=0.5)
[Epoch 11] Combined AUPRC: 0.6299 (r=0.5)
[Epoch 12] Combined AUPRC: 0.6421 (r=0.5)
[Epoch 13] Combined AUPRC: 0.6258 (r=0.5)
[Epoch 14] Combined AUPRC: 0.6387 (r=0.5)
[Epoch 15] Combined AUPRC: 0.6268 (r=0.5)
[Epoch 16] Combined AUPRC: 0.6389 (r=0.5)
[Epoch 17] Combined AUPRC: 0.6381 (r=0.5)
[Epoch 18] Combined AUPRC: 0.6384 (r=0.5)
[Epoch 19] Combined AUPRC: 0.6290 (r=0.5)
[Epoch 20] Combined AUPRC: 0.6168 (r=0.5)
[Epoch 21] Combined AUPRC: 0.6424 (r=0.5)
[Epoch 22] Combined AUPRC: 0.6336 (r=0.5)
[Epoch 23] Combined AUPRC: 0.6435 (r=0.5)
[E